# Chapter 4: Simulation on Normal Populations


# Part I Training process

Only minimum requirements for the training of the CNNs for solving the Normal Populations

## Setup

In [1]:
import numpy as np
from scipy import stats   #ttest

In [2]:
# Mount in drive:
# Just first time:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Functions

In [3]:
# 2) Function:
def Normal_Population_Simulation(B = 2,
                                 m = 10,
                                 p0 = 0.9,
                                 N = 2,
                                 mu_A = 2,
                                 seed = 42,
                                 reduced = True):
  '''
  Simulation of Normal populations:

  1) Inputs:
  -----------

  2) Outputs:
  -----------

  '''
  
  # Control population: All mean 0
  np.random.seed(seed)
  control = np.random.normal(loc = 0.0, scale=1.0, size = (B,m,N))

  m0 = np.int16(np.floor(m*p0))  # Number of test where null hypothesis is true (s)
  m1 = np.int16(m-m0)            # Number of test where null hypothesis is false (saved like int)
  null_Hypothesis = np.repeat(0,m)
  mx = np.repeat(0,m)
  null_Hypothesis[0:m1] = 1
  mx[0:m1] = mu_A

  v_mx = np.array([])
  # Alternative population
  for i in range(B):
    v_mx = np.append(v_mx, np.repeat(mx, N))

  np.random.seed(seed + 1)
  alternative_pop = np.random.normal(loc = v_mx, scale = 1.0).reshape(B,m,N)

  set_pvalues = np.zeros(shape = (B, m))
  set_o_index = np.zeros(shape = (B, m))
  set_Null_H  = np.zeros(shape = (B, m))
  set_lbbf    = np.zeros(shape = (B, m))
  set_images  = np.zeros(shape = (B, m, m))

  div_lbbf = lambda t: lbbf[t]/lbbf

  for index in range(B):
    # Original distribution null_hypothesis
    code_nullHypothesis = null_Hypothesis.copy()
    
    p_values = np.array([result[1] for result in list(map(stats.ttest_ind, control[index,:,:], alternative_pop[index,:,:]))])
    o = np.argsort(p_values)
    set_o_index[index,:] = o
    # Order p-values and code_nullHypothesis
    p_values = p_values[o]
    set_pvalues[index,:] = p_values
    code_nullHypothesis  = code_nullHypothesis[o]
    set_Null_H[index, :] = code_nullHypothesis
    
    # LBBF
    lbbf = np.where(p_values < (1/np.exp(1)), -np.exp(1)*p_values*np.log(p_values), 1)
    set_lbbf[index,:] = lbbf

    # Image
    image_lbbf = np.array([div_lbbf(xi) for xi in np.arange(0,m)]).T

    # We compute the maximum lbf quotient, in order to standarize the matrix in the range 0 and 1
    max_lbbf = np.max(image_lbbf)

    # Input of the ANNs - CNNs
    set_images[index,:,:] = image_lbbf/max_lbbf
    #End for index in range(B):

  # otuput of the function
  if reduced is True:
    #Only X, Y and p-values
    output_dict = {'X': set_images,  'Y': set_Null_H, 
                  'p_value': set_pvalues}
  else:
    # Add LBBF, index, mu_A and p0
    output_dict = {'X': set_images,  'Y': set_Null_H, 
                  'p_value': set_pvalues,  'LBBF':  set_lbbf, 
                  'index': set_o_index,
                  'mu_A': mu_A, 'p0': p0, 'N':N }

  return output_dict

## CNN training

### Training I-II: m = 50, 100, N = 20

In [ ]:
def generate_global_training(N,
                             m,
                             B,
                             v_mu_A,
                             v_p0s,
                             seed = 42):


  # 1) Generate the Global training data
  global_training_I = [Normal_Population_Simulation(B = B,
                                                N = N,
                                                m = m, 
                                                p0 = _p0s,
                                                mu_A = _mu_A,
                                                seed = seed) 
                    for _p0s in p0s
                    for _mu_A in mu_A]
  
  return global_training_I

In [ ]:
def global_training_split(global_training_I, seed = 42):

  # Number of iterations
  n_sim_gt_I = len(global_training_I)

  # Number of simulations:
  n_sim_gt_I_B = n_sim_gt_I*B

  # Extracting the image as X_train
  X_gt_I = np.array([global_training_I[index]['X'] for index in range(0, n_sim_gt_I)])
  X_gt_I = X_gt_I.reshape(n_sim_gt_I_B, m, m)

  Y_gt_I = np.array([global_training_I[index]['Y'] for index in range(0, n_sim_gt_I)])
  Y_gt_I = Y_gt_I.reshape(n_sim_gt_I_B, m)

  # Separtion train(validation)/test using permutation: train_test_split consumes a lot of memory
  np.random.seed(seed)
  total_random_sample_I = np.random.permutation(n_sim_gt_I_B)

  ## Train/test
  n_test_I = np.int16(np.floor(n_sim_gt_I_B*0.2))
  n_train_full_I = n_sim_gt_I_B - n_test_I

  idx_train_full_I = total_random_sample_I[:n_train_full_I]
  idx_test_I = total_random_sample_I[n_train_full_I:]

  ## Validation/test
  n_val_I = np.int16(np.floor(n_train_full_I*0.1))
  n_train_I = n_train_full_I - n_val_I

  idx_train_I = total_random_sample_I[:n_train_I]
  idx_valid_I = total_random_sample_I[(n_train_full_I - n_val_I):n_train_full_I]

  X_train_I, X_valid_I, X_test_I = X_gt_I[idx_train_I,:,:], X_gt_I[idx_valid_I,:,:], X_gt_I[idx_test_I,:,:]

  # No. of channels = 1 (gray scale)
  X_train_I = X_train_I[..., np.newaxis]
  X_valid_I = X_valid_I[..., np.newaxis]
  X_test_I  = X_test_I[..., np.newaxis]

  # Response
  y_train_I, y_valid_I, y_test_I = Y_gt_I[idx_train_I,:], Y_gt_I[idx_valid_I,:], Y_gt_I[idx_test_I,:]

  return X_train_I, X_valid_I, X_test_I, y_train_I, y_valid_I, y_test_I

In [ ]:
# Define a simple sequential model
def create_CNN_model(INPUT_SHAPE,
                     OUT_CLASSES,
                     INITIALIZER,
                     NEURONS_1, NEURONS_2,
                     DROPOUT,
                     OPTIMIZER,
                     METRICS,
                     SEED = 42):

  # 0)
  keras.backend.clear_session()

  # 1) Set seed
  np.random.seed(SEED)
  tf.random.set_seed(SEED)

  # 2) Sequential model
  model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 2, 
                        kernel_size = (2,2),
                        strides = (1,1),
                        padding = "same", 
                        activation = "relu",
                        kernel_initializer = INITIALIZER,
                        input_shape = INPUT_SHAPE),
      keras.layers.MaxPooling2D(pool_size=(2, 2), 
                              strides=(2,2), 
                              padding='same'), 
      keras.layers.Flatten(),
      keras.layers.Dense(NEURONS_1, 
                       activation="relu", 
                       kernel_initializer = INITIALIZER),             
      keras.layers.Dense(NEURONS_2, 
                       activation="relu", 
                       kernel_initializer = INITIALIZER),
      keras.layers.Dropout(DROPOUT),
      keras.layers.Dense(OUT_CLASSES, 
                         activation = "sigmoid", 
                         kernel_initializer = INITIALIZER)                                 
  ])

  #2) compile method
  model.compile(loss      = 'binary_crossentropy',
                optimizer = OPTIMIZER,
                metrics   = METRICS)
  
  return model

In [ ]:
# 1) Basic Features:

m = 50    # Number of features
N = 20     # Number of sample units
B = 200    # Number of Simulations

# 2) Simulated characteristics

## Mean Alternative
mu_A = np.arange(0.3, 2.5, 0.1)

## Proportion true Null Hypothesis
p0s = np.arange(0.9, 1, 0.01)

## Print Zone:
print('Basic Features:')
print('\nNumber of features: m =', m)
print('Number of Sample Units: N =', N)
print('Number of simulations: B =', B)
print('------------------------------')
print('Simulated characteristics:')
print('\n Alternative mean')
print(mu_A)
print(mu_A.shape)
print('----------------')
print('\n Proportion True Null Hypothesis')
print(p0s)
print(p0s.shape)

Basic Features:

Number of features: m = 50
Number of Sample Units: N = 20
Number of simulations: B = 200
------------------------------
Simulated characteristics:

 Alternative mean
[0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.
 2.1 2.2 2.3 2.4]
(22,)
----------------

 Proportion True Null Hypothesis
[0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99]
(10,)


In [ ]:
global_training_m50_N20 = generate_global_training(N = N,
                             m = m,
                             B = B,
                             v_mu_A = mu_A,
                             v_p0s = p0s,
                             seed = 8)

In [ ]:
import pickle
file_name = "global_training_m50_N20.pkl"

open_file = open(file_name, "wb")
pickle.dump(global_training_m50_N20, open_file)
open_file.close()

# directory
import os
print( os.getcwd() )
print( os.listdir() )



/content
['.config', 'gdrive', 'global_training_m50_N20.pkl', 'sample_data']


In [ ]:
%cp /content/global_training_m50_N20.pkl /content/gdrive/My\ Drive

In [ ]:
X_train_I, X_valid_I, X_test_I, y_train_I, y_valid_I, y_test_I = global_training_split(global_training_m50_N20)

## Print dimensions for inputs
print('Inputs')
print('X_train shape:', X_train_I.shape)
print('X_valid shape:', X_valid_I.shape)
print('X_test shape: ', X_test_I.shape)
print('--------------------------')

print('\nResponse')
print(y_train_I.shape)
print(y_valid_I.shape)
print(y_test_I.shape)

Inputs
X_train shape: (31680, 50, 50, 1)
X_valid shape: (3520, 50, 50, 1)
X_test shape:  (8800, 50, 50, 1)
--------------------------

Response
(31680, 50)
(3520, 50)
(8800, 50)


In [ ]:
%who_ls

['B',
 'DROPOUT',
 'EPOCHS',
 'INITIALIZER',
 'INPUT_SHAPE',
 'METRICS',
 'N',
 'NEURONS_1',
 'NEURONS_2',
 'Normal_Population_Simulation',
 'OPTIMIZER',
 'OUT_CLASSES',
 'X_test_I',
 'X_train_I',
 'X_valid_I',
 'create_CNN_model',
 'drive',
 'early_stopping',
 'eval_test_I',
 'eval_train_I',
 'file_name',
 'generate_global_training',
 'global_training_split',
 'history_CNN_I',
 'keras',
 'm',
 'model_CNN_I',
 'mu_A',
 'np',
 'open_file',
 'os',
 'p0s',
 'pickle',
 'start_time',
 'stats',
 'tf',
 'time',
 'y_test_I',
 'y_train_I',
 'y_valid_I']

In [ ]:
del global_training_m50_N20

In [ ]:
# Import tensorFlow / Keras
import tensorflow as tf
from tensorflow import keras


In [ ]:
INPUT_SHAPE  = [m, m, 1]
OUT_CLASSES = m

# 2) Hyperparameters:

## 2.1) Number of neurons by (hidden) layer
NEURONS_1 = 500   # of the total pixels
NEURONS_2 = 250   #  of the total pixels
DROPOUT = 0.5

## 2.2) Initial weight
INITIALIZER = keras.initializers.HeUniform()

# 3)  compilationHyperparameters
OPTIMIZER = keras.optimizers.Nadam()
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='AUROC'),
      keras.metrics.AUC(name='AUPRC', curve='PR'), # precision-recall curve
]

# Create the model instance
model_CNN_I = create_CNN_model(INPUT_SHAPE,
                     OUT_CLASSES,
                     INITIALIZER,
                     NEURONS_1, NEURONS_2,
                     DROPOUT,
                     OPTIMIZER,
                     METRICS)

# Print Model summary (model's architecture)
model_CNN_I.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 2)         10        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 2)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1250)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               625500    
_________________________________________________________________
dense_1 (Dense)              (None, 250)               125250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                1

In [ ]:
# Set the correct initial bias
eval_train_I = model_CNN_I.evaluate(X_train_I, y_train_I, verbose=0)

# Print loss: Correct if loss is greater > 1
print('Train loss = {:.3f}'.format(eval_train_I[0]))
print('-----------------------')
print('Example predictions:')
model_CNN_I.predict(X_train_I[0:1,])


# Model Train

# 1) Hypeparameters
EPOCHS = 50

# Early_stopping (avoid overfitting)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_AUPRC', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# 2) fit method
import time
start_time = time.time()
history_CNN_I = model_CNN_I.fit(X_train_I, 
                            y_train_I, 
                            epochs = EPOCHS,
                            validation_data = (X_valid_I, y_valid_I),
                            callbacks=[early_stopping],
                            verbose=0)
print('Elapsed time (seconds) = {:.3f}'.format(time.time() - start_time))

Train loss = 0.703
-----------------------
Example predictions:
Elapsed time (seconds) = 1749.167


In [ ]:
eval_train_I = model_CNN_I.evaluate(X_train_I, y_train_I, verbose=0)

# Print loss and metrics evaluation train set
print('Train loss = {:.3f}'.format(eval_train_I[0]))
print('-----------------------')
print('Train Accuracy  = {:.3f}'.format(eval_train_I[1]))
print('Train Precision = {:.3f}'.format(eval_train_I[2]))
print('Train Recall    = {:.3f}'.format(eval_train_I[3]))
print('Train AUROC     = {:.3f}'.format(eval_train_I[4]))
print('Train AUPRC     = {:.3f}'.format(eval_train_I[5]))

Train loss = 0.038
-----------------------
Train Accuracy  = 0.983
Train Precision = 0.933
Train Recall    = 0.776
Train AUROC     = 0.996
Train AUPRC     = 0.953


In [ ]:
eval_test_I = model_CNN_I.evaluate(X_test_I, y_test_I, verbose = 0)

# Print loss and metrics evaluation test set
print('Test loss = {:.3f}'.format(eval_test_I[0]))
print('-----------------------')
print('Test Accuracy  = {:.3f}'.format(eval_test_I[1]))
print('Test Precision = {:.3f}'.format(eval_test_I[2]))
print('Test Recall    = {:.3f}'.format(eval_test_I[3]))
print('Test AUROC     = {:.3f}'.format(eval_test_I[4]))
print('Test AUPRC     = {:.3f}'.format(eval_test_I[5]))

Test loss = 0.040
-----------------------
Test Accuracy  = 0.982
Test Precision = 0.930
Test Recall    = 0.764
Test AUROC     = 0.996
Test AUPRC     = 0.948


In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

# Save:
model_CNN_I.save('my_model_Normal_m50_N20.h5') 

%cp /content/my_model_Normal_m50_N20.h5 /content/gdrive/My\ Drive

### Model with m = 100

In [ ]:
# 1) Basic Features:

m = 100    # Number of features
N = 20     # Number of sample units
B = 200    # Number of Simulations

# 2) Simulated characteristics

## Mean Alternative
mu_A = np.arange(0.3, 2.5, 0.1)

## Proportion true Null Hypothesis
p0s = np.arange(0.9, 1, 0.01)

## Print Zone:
print('Basic Features:')
print('\nNumber of features: m =', m)
print('Number of Sample Units: N =', N)
print('Number of simulations: B =', B)
print('------------------------------')
print('Simulated characteristics:')
print('\n Alternative mean')
print(mu_A)
print(mu_A.shape)
print('----------------')
print('\n Proportion True Null Hypothesis')
print(p0s)
print(p0s.shape)


global_training_m100_N20 = generate_global_training(N = N,
                             m = m,
                             B = B,
                             v_mu_A = mu_A,
                             v_p0s = p0s,
                             seed = 8)
							 
print('Number of iterations:', len(global_training_m100_N20))

Basic Features:

Number of features: m = 100
Number of Sample Units: N = 20
Number of simulations: B = 200
------------------------------
Simulated characteristics:

 Alternative mean
[0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.
 2.1 2.2 2.3 2.4]
(22,)
----------------

 Proportion True Null Hypothesis
[0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99]
(10,)
Number of iterations: 220


In [ ]:
# Save:
import pickle
file_name = "global_training_m100_N20.pkl"

open_file = open(file_name, "wb")
pickle.dump(global_training_m100_N20, open_file)
open_file.close()

# directory
import os
print( os.getcwd() )
print( os.listdir() )

/content
['.config', 'my_model_Normal_m50_N20.h5', 'global_training_m100_N20.pkl', 'gdrive', 'global_training_m50_N20.pkl', 'sample_data']


In [ ]:
%cp /content/global_training_m100_N20.pkl /content/gdrive/My\ Drive

In [ ]:
%who_ls

['B',
 'N',
 'Normal_Population_Simulation',
 'create_CNN_model',
 'drive',
 'file_name',
 'generate_global_training',
 'global_training_m100_N20',
 'global_training_split',
 'm',
 'mu_A',
 'np',
 'open_file',
 'os',
 'p0s',
 'pickle',
 'stats']

In [ ]:
X_train_I, X_valid_I, X_test_I, y_train_I, y_valid_I, y_test_I = global_training_split(global_training_m100_N20)


## Print dimensions for inputs
print('Inputs')
print('X_train shape:', X_train_I.shape)
print('X_valid shape:', X_valid_I.shape)
print('X_test shape: ', X_test_I.shape)
print('--------------------------')

print('\nResponse')
print(y_train_I.shape)
print(y_valid_I.shape)
print(y_test_I.shape)

Inputs
X_train shape: (31680, 100, 100, 1)
X_valid shape: (3520, 100, 100, 1)
X_test shape:  (8800, 100, 100, 1)
--------------------------

Response
(31680, 100)
(3520, 100)
(8800, 100)


In [ ]:
del global_training_m100_N20

In [ ]:
INPUT_SHAPE  = [m, m, 1]
OUT_CLASSES = m

# 2) Hyperparameters:

## 2.1) Number of neurons by (hidden) layer
NEURONS_1 = 500   # of the total pixels
NEURONS_2 = 250   #  of the total pixels
DROPOUT = 0.5

## 2.2) Initial weight
INITIALIZER = keras.initializers.HeUniform()

# 3)  compilationHyperparameters
OPTIMIZER = keras.optimizers.Nadam()
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='AUROC'),
      keras.metrics.AUC(name='AUPRC', curve='PR'), # precision-recall curve
]

# Create the model instance
model_CNN_I = create_CNN_model(INPUT_SHAPE,
                     OUT_CLASSES,
                     INITIALIZER,
                     NEURONS_1, NEURONS_2,
                     DROPOUT,
                     OPTIMIZER,
                     METRICS)

# Print Model summary (model's architecture)
model_CNN_I.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 2)       10        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 2)         0         
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               2500500   
_________________________________________________________________
dense_1 (Dense)              (None, 250)               125250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [ ]:
# Set the correct initial bias
eval_train_I = model_CNN_I.evaluate(X_train_I, y_train_I, verbose=0)

# Print loss: Correct if loss is greater > 1
print('Train loss = {:.3f}'.format(eval_train_I[0]))
print('-----------------------')
print('Example predictions:')
model_CNN_I.predict(X_train_I[0:1,])


# Model Train

# 1) Hypeparameters
EPOCHS = 50

# Early_stopping (avoid overfitting)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_AUPRC', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# 2) fit method
import time
start_time = time.time()
history_CNN_I = model_CNN_I.fit(X_train_I, 
                            y_train_I, 
                            epochs = EPOCHS,
                            validation_data = (X_valid_I, y_valid_I),
                            callbacks=[early_stopping],
                            verbose=0)
print('Elapsed time (seconds) = {:.3f}'.format(time.time() - start_time))

Train loss = 0.697
-----------------------
Example predictions:
Elapsed time (seconds) = 5425.959


In [ ]:
# directory
import os
print( os.getcwd() )
print( os.listdir() )


/content
['.config', 'my_model_Normal_m50_N20.h5', 'global_training_m100_N20.pkl', 'gdrive', 'global_training_m50_N20.pkl', 'sample_data']


In [ ]:
del my_model_Normal_m100_N20.h5

NameError: ignored

In [ ]:
# Save:
model_CNN_I.save('my_model_Normal_m100_N20.h5') 

%cp /content/my_model_Normal_m100_N20.h5 /content/gdrive/My\ Drive

### Training III: m = 150, 100, N = 20

In [4]:
# 1) Basic Features:

m = 150    # Number of features
N = 20     # Number of sample units
B = 150    # Number of Simulations

# 2) Simulated characteristics

## Mean Alternative
mu_A = np.arange(0.3, 2.5, 0.1)

## Proportion true Null Hypothesis
p0s = np.arange(0.9, 1, 0.01)

## Print Zone:
print('Basic Features:')
print('\nNumber of features: m =', m)
print('Number of Sample Units: N =', N)
print('Number of simulations: B =', B)
print('------------------------------')
print('Simulated characteristics:')
print('\n Alternative mean')
print(mu_A)
print(mu_A.shape)
print('----------------')
print('\n Proportion True Null Hypothesis')
print(p0s)
print(p0s.shape)

Basic Features:

Number of features: m = 150
Number of Sample Units: N = 20
Number of simulations: B = 150
------------------------------
Simulated characteristics:

 Alternative mean
[0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.
 2.1 2.2 2.3 2.4]
(22,)
----------------

 Proportion True Null Hypothesis
[0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99]
(10,)


In [5]:
import time
start_time = time.time()
global_training_I = [Normal_Population_Simulation(B = B,
                                                N = N,
                                                m = m, 
                                                p0 = _p0s,
                                                mu_A = _mu_A,
                                                seed = 8) 
                   for _p0s in p0s
                   for _mu_A in mu_A]
print('Elapsed time (seconds) = {:.3f}'.format(time.time() - start_time))

# Number of iterations
n_sim_gt_I = len(global_training_I)

print('Number of iterations:', n_sim_gt_I)

Elapsed time (seconds) = 1749.300
Number of iterations: 220


### Savel/Load  data set

Save the simulated object.

In [6]:
import pickle
file_name = "global_training_m150_N20.pkl"

open_file = open(file_name, "wb")
pickle.dump(global_training_I, open_file)
open_file.close()

In [7]:
# directory
import os
print( os.getcwd() )
print( os.listdir() )

/content
['.config', 'global_training_m150_N20.pkl', 'gdrive', 'sample_data']


In [8]:
# Mount in drive:
# Just first time:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
%cp /content/global_training_m150_N20.pkl /content/gdrive/My\ Drive

In [ ]:
# load
import pickle

DATA_PATH = '/content/gdrive/MyDrive'
infile = open(DATA_PATH+'/global_training_m50_N20.pkl','rb')
global_training = pickle.load(infile)


### Training Process

In [10]:
# Number of simulations:
n_sim_gt_I_B = n_sim_gt_I*B

print('Number of simulations:' , n_sim_gt_I_B)
print('------------------------------')

# Extracting the image as X_train
X_gt_I = np.array([global_training_I[index]['X'] for index in range(0, n_sim_gt_I)])
X_gt_I = X_gt_I.reshape(n_sim_gt_I_B, m, m)
print('X shape:', X_gt_I.shape)

Y_gt_I = np.array([global_training_I[index]['Y'] for index in range(0, n_sim_gt_I)])
Y_gt_I = Y_gt_I.reshape(n_sim_gt_I_B, m)
print('Y shape:', Y_gt_I.shape)

Number of simulations: 33000
------------------------------
X shape: (33000, 150, 150)
Y shape: (33000, 150)


In [12]:
# List of active objects
%who_ls

['B',
 'N',
 'Normal_Population_Simulation',
 'X_gt_I',
 'Y_gt_I',
 'drive',
 'file_name',
 'm',
 'mu_A',
 'n_sim_gt_I',
 'n_sim_gt_I_B',
 'np',
 'open_file',
 'os',
 'p0s',
 'pickle',
 'start_time',
 'stats',
 'time']

In [11]:
# Delete global_training_I for liberating RAM
del global_training_I

In [13]:
# Separtion train(validation)/test using permutation: train_test_split consumes a lot of memory
np.random.seed(42)
total_random_sample_I = np.random.permutation(n_sim_gt_I_B)

## Train/test
n_test_I = np.int16(np.floor(n_sim_gt_I_B*0.2))
n_train_full_I = n_sim_gt_I_B - n_test_I

idx_train_full_I = total_random_sample_I[:n_train_full_I]
idx_test_I = total_random_sample_I[n_train_full_I:]

## Validation/test
n_val_I = np.int16(np.floor(n_train_full_I*0.1))
n_train_I = n_train_full_I - n_val_I

idx_train_I = total_random_sample_I[:n_train_I]
idx_valid_I = total_random_sample_I[(n_train_full_I - n_val_I):n_train_full_I]

print('Index: train, validation, test:')
print(n_train_I,n_val_I, n_test_I)


Index: train, validation, test:
23760 2640 6600


In [14]:
X_train_I, X_valid_I, X_test_I = X_gt_I[idx_train_I,:,:], X_gt_I[idx_valid_I,:,:], X_gt_I[idx_test_I,:,:]

# No. of channels = 1 (gray scale)

X_train_I = X_train_I[..., np.newaxis]
X_valid_I = X_valid_I[..., np.newaxis]
X_test_I  = X_test_I[..., np.newaxis]

## Print dimensions for inputs
print('Inputs')
print('X_train shape:', X_train_I.shape)
print('X_valid shape:', X_valid_I.shape)
print('X_test shape: ', X_test_I.shape)
print('--------------------------')


# Response
y_train_I, y_valid_I, y_test_I = Y_gt_I[idx_train_I,:], Y_gt_I[idx_valid_I,:], Y_gt_I[idx_test_I,:]

## For studying the proportions train/test
#y_train_full = Y[idx_train_full,:]
print('\nResponse')
print(y_train_I.shape)
print(y_valid_I.shape)
print(y_test_I.shape)

Inputs
X_train shape: (23760, 150, 150, 1)
X_valid shape: (2640, 150, 150, 1)
X_test shape:  (6600, 150, 150, 1)
--------------------------

Response
(23760, 150)
(2640, 150)
(6600, 150)


In [ ]:
%who_ls

['B',
 'N',
 'Normal_Population_Simulation',
 'X_test_I',
 'X_train_I',
 'X_valid_I',
 'drive',
 'file_name',
 'm',
 'mu_A',
 'np',
 'open_file',
 'os',
 'p0s',
 'pickle',
 'start_time',
 'stats',
 'time',
 'total_random_sample_I',
 'y_test_I',
 'y_train_I',
 'y_valid_I']

In [15]:
# Delete innecesary objects (Free RAM)
del X_gt_I,Y_gt_I, idx_test_I,idx_train_I,idx_train_full_I,idx_valid_I,
del n_sim_gt_I,n_sim_gt_I_B,n_test_I,n_train_I,n_train_full_I,n_val_I

In [16]:
# Multilabel classification: Define the number of elements of input/output in the CNN

INPUT_SHAPE  = [m, m, 1]
OUT_CLASSES = m

# print
print('Input: Dimension image LBBF: ', INPUT_SHAPE)
print('Output: Number of labels', OUT_CLASSES)

Input: Dimension image LBBF:  [150, 150, 1]
Output: Number of labels 150


In [17]:
# Define a simple sequential model
def create_CNN_model(INPUT_SHAPE,
                     OUT_CLASSES,
                     INITIALIZER,
                     NEURONS_1, NEURONS_2,
                     DROPOUT,
                     OPTIMIZER,
                     METRICS,
                     SEED = 42):

  # 0)
  keras.backend.clear_session()

  # 1) Set seed
  np.random.seed(SEED)
  tf.random.set_seed(SEED)

  # 2) Sequential model
  model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 2, 
                        kernel_size = (2,2),
                        strides = (1,1),
                        padding = "same", 
                        activation = "relu",
                        kernel_initializer = INITIALIZER,
                        input_shape = INPUT_SHAPE),
      keras.layers.MaxPooling2D(pool_size=(2, 2), 
                              strides=(2,2), 
                              padding='same'), 
      keras.layers.Flatten(),
      keras.layers.Dense(NEURONS_1, 
                       activation="relu", 
                       kernel_initializer = INITIALIZER),             
      keras.layers.Dense(NEURONS_2, 
                       activation="relu", 
                       kernel_initializer = INITIALIZER),
      keras.layers.Dropout(DROPOUT),
      keras.layers.Dense(OUT_CLASSES, 
                         activation = "sigmoid", 
                         kernel_initializer = INITIALIZER)                                 
  ])

  #2) compile method
  model.compile(loss      = 'binary_crossentropy',
                optimizer = OPTIMIZER,
                metrics   = METRICS)
  
  return model

In [18]:
# Import tensorFlow / Keras
import tensorflow as tf
from tensorflow import keras

In [19]:
# 2) Hyperparameters:

## 2.1) Number of neurons by (hidden) layer
NEURONS_1 = 500   # of the total pixels
NEURONS_2 = 250   #  of the total pixels
DROPOUT = 0.5

## 2.2) Initial weight
INITIALIZER = keras.initializers.HeUniform()

# 3)  compilationHyperparameters
LOSS_FN   = keras.losses.binary_crossentropy
OPTIMIZER = keras.optimizers.Nadam()
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='AUROC'),
      keras.metrics.AUC(name='AUPRC', curve='PR'), # precision-recall curve
]

# Create the model instance
model_CNN_I = create_CNN_model(INPUT_SHAPE,
                     OUT_CLASSES,
                     INITIALIZER,
                     NEURONS_1, NEURONS_2,
                     DROPOUT,
                     OPTIMIZER,
                     METRICS)

# Print Model summary (model's architecture)
model_CNN_I.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 2)       10        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 2)         0         
_________________________________________________________________
flatten (Flatten)            (None, 11250)             0         
_________________________________________________________________
dense (Dense)                (None, 500)               5625500   
_________________________________________________________________
dense_1 (Dense)              (None, 250)               125250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               3

In [20]:
# Set the correct initial bias
eval_train_I = model_CNN_I.evaluate(X_train_I, y_train_I, verbose=0)

# Print loss: Correct if loss is greater > 1
print('Train loss = {:.3f}'.format(eval_train_I[0]))
print('-----------------------')
print('Example predictions:')
model_CNN_I.predict(X_train_I[0:1,])

Train loss = 0.696
-----------------------
Example predictions:


array([[0.42153144, 0.51392585, 0.52650195, 0.48152453, 0.45427305,
        0.5377739 , 0.5003329 , 0.55866593, 0.52481467, 0.51236296,
        0.5341357 , 0.51553494, 0.48535153, 0.4794514 , 0.514196  ,
        0.4618243 , 0.49519786, 0.5060526 , 0.52056104, 0.5096314 ,
        0.5114958 , 0.47163588, 0.5219136 , 0.5547713 , 0.4554237 ,
        0.4482477 , 0.47268584, 0.45575017, 0.55727416, 0.54937565,
        0.41895354, 0.5107314 , 0.51370573, 0.46989584, 0.4574647 ,
        0.5297797 , 0.51061434, 0.51526344, 0.53976977, 0.54871535,
        0.51163995, 0.50638163, 0.5525162 , 0.44618475, 0.50964165,
        0.5166262 , 0.4869049 , 0.4856019 , 0.44208226, 0.5523695 ,
        0.47969216, 0.5396057 , 0.43900704, 0.5035232 , 0.5771238 ,
        0.54697883, 0.52505445, 0.5348586 , 0.51057476, 0.4216159 ,
        0.51973206, 0.504148  , 0.49881312, 0.5339632 , 0.5625825 ,
        0.515063  , 0.47596818, 0.4886967 , 0.4978951 , 0.41676232,
        0.48424125, 0.5436522 , 0.52866447, 0.51

In [21]:
del eval_train_I

In [22]:
# Model Train

# 1) Hypeparameters
EPOCHS = 50

# Early_stopping (avoid overfitting)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_AUPRC', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# 2) fit method
import time
start_time = time.time()
history_CNN_I = model_CNN_I.fit(X_train_I, 
                            y_train_I, 
                            epochs = EPOCHS,
                            validation_data = (X_valid_I, y_valid_I),
                            callbacks=[early_stopping],
                            verbose=0)
print('Elapsed time (seconds) = {:.3f}'.format(time.time() - start_time))

Elapsed time (seconds) = 1145.991


### Save/Load model

In [23]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [24]:
# Save:
model_CNN_I.save('my_model_Normal_m150_N20.h5') 

%cp /content/my_model_Normal_m150_N20.h5 /content/gdrive/My\ Drive

#from google.colab import files
#files.download("my_model_Normal_m50_N20.h5")

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
model_CNN_I = keras.models.load_model('/content/gdrive/MyDrive/Normal_CNN_N_20_m_100.h5')

# Show the model architecture
model_CNN_I.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 2)       10        
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 500)               10000500  
_________________________________________________________________
dense_1 (Dense)              (None, 250)               125250    
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               25100     
Total params: 10,150,860
Trainable params: 10,150,860
Non-trainable params: 0
____________________________________________

In [ ]:
eval_train_I = model_CNN_I.evaluate(X_train_I, y_train_I, verbose=0)

# Print loss and metrics evaluation train set
print('Train loss = {:.3f}'.format(eval_train_I[0]))
print('-----------------------')
print('Train Accuracy  = {:.3f}'.format(eval_train_I[1]))
print('Train Precision = {:.3f}'.format(eval_train_I[2]))
print('Train Recall    = {:.3f}'.format(eval_train_I[3]))
print('Train AUROC     = {:.3f}'.format(eval_train_I[4]))
print('Train AUPRC     = {:.3f}'.format(eval_train_I[5]))

In [ ]:
eval_test_I = model_CNN_I.evaluate(X_test_I, y_test_I, verbose = 0)

# Print loss and metrics evaluation test set
print('Test loss = {:.3f}'.format(eval_test_I[0]))
print('-----------------------')
print('Test Accuracy  = {:.3f}'.format(eval_test_I[1]))
print('Test Precision = {:.3f}'.format(eval_test_I[2]))
print('Test Recall    = {:.3f}'.format(eval_test_I[3]))
print('Test AUROC     = {:.3f}'.format(eval_test_I[4]))
print('Test AUPRC     = {:.3f}'.format(eval_test_I[5]))

Test loss = 0.034
-----------------------
Test Accuracy  = 0.986
Test Precision = 0.913
Test Recall    = 0.837
Test AUROC     = 0.997
Test AUPRC     = 0.958
